![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Dupire model
$C\left(S_0, K, T\right)=\int_K^{\infty} d S_T \varphi\left(S_T, T ; S_0\right)\left(S_T-K\right)$
### risk-neutral density function φ of the final spot $S_T$
$\varphi\left(K, T ; S_0\right)=\frac{\partial^2 C}{\partial K^2}$
## Drift
$\mu = r_t − D_t$
## local volatility
$\sigma^2\left(K, T, S_0\right)=\frac{\frac{\partial C}{\partial T}}{\frac{1}{2} K^2 \frac{\partial^2 C}{\partial K^2}}$
## undiscounted option price C in terms of the strike price K:
### which is the Dupire equation when the underlying stock has risk-neutral drift μ.
$\frac{\partial C}{\partial T}=\frac{\sigma^2 K^2}{2} \frac{\partial^2 C}{\partial K^2}+\left(r_t-D_t\right)\left(C-K \frac{\partial C}{\partial K}\right)$
# BSM model
$Call=S  N\left(d_1\right)-K e^{-r \tau} N\left(d_2\right)$ 
\
$d_1=\frac{\ln (S / K)+\left(r-y+\sigma^2 / 2\right) \tau}{\sigma \sqrt{\tau}}$
\
$d_2=\frac{\ln (S / K)+\left(r-y-\sigma^2 / 2\right) \tau}{\sigma \sqrt{\tau}}=d_1-\sigma \sqrt{\tau}$

## PDF of Normal  
$f(x)=\frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$

## CDF of Normal
$N(x)=\frac{1}{\sqrt{2 \pi}} \int_{-\infty}^x e^{-z^2 / 2} d z$


# Importing Libraries

In [2]:
import sympy as smp 
import numpy as np
from sympy.stats import P, E, variance, Die, Normal
from sympy import simplify



# Risk-neutral density function φ of the final spot $S_T$
$\varphi\left(K, T ; S_0\right)=\frac{\partial^2 C}{\partial K^2}$

In [4]:
#define the function symbols
Call,varphi,N = smp.symbols('Call varphi N', cls=smp.Function) 
S_0,K,T,S_T,D,r,sigma,tau ,d_1,d_2 = smp.symbols('S_0 K T S_T D r sigma tau d_1 d_2',real=True) 


Call = smp.symbols('Call', cls=smp.Function) 
f_d_1,f_d_2,F_d_1,F_d_2 = smp.symbols('f_d_1 f_d_2 F_d_1 F_d_2' , cls=smp.Function) 
mu_google,sigma_google= smp.symbols('mu_google sigma_google',real=True) 
f_d_1 = Normal('d_1', mu_google, sigma_google)
f_d_2 = Normal('d_2', mu_google, sigma_google)


F_d_1 = simplify(P(f_d_1>tau))
F_d_2 = simplify(P(f_d_2>tau))
Call = S_T  * F_d_1 - K * smp.exp(-r*tau) * F_d_2


varphi = smp.diff(Call,K,2)

#varphi.subs([(Call,Call),(K,int(K))]).evalf()


## Undiscounted option price C in terms of the strike price K:
### which is the Dupire equation when the underlying stock has risk-neutral drift μ.
$\frac{\partial C}{\partial T}=\frac{\sigma^2 K^2}{2} \frac{\partial^2 C}{\partial K^2}+\left(r_t-D_t\right)\left(C-K \frac{\partial C}{\partial K}\right)$

In [5]:
dCdT = sigma**2 * K / 2 * varphi + (r - D)* (Call - K * smp.diff(Call,K))
dCdT

S_T*(-D + r)*Piecewise((erf(sqrt(2)*(mu_google - tau)/(2*sigma_google))/2 + 1/2, (Abs(arg(sigma_google)) < pi/4) | ((Abs(arg(sigma_google)) <= pi/4) & (Abs(2*arg(mu_google) - 4*arg(sigma_google) + 2*arg((mu_google - tau)/mu_google) + 2*pi) < pi))), (sqrt(2)*Integral(exp(-(_z - mu_google + tau)**2/(2*sigma_google**2)), (_z, 0, oo))/(2*sqrt(pi)*sigma_google), True))

## Local volatility
$\sigma^2\left(K, T, S_0\right)=\frac{\frac{\partial C}{\partial T}}{\frac{1}{2} K^2 \frac{\partial^2 C}{\partial K^2}}$

The right-hand side of this equation can be computed from known European option prices. So, given a complete set of European option prices for all strikes and expirations, local volatilities are given uniquely by equation above.
We can view this equation as a definition of the local volatility function regardless of what kind of process (stochastic volatility for example) actually governs the evolution of volatility.

In [6]:
local_volatility = smp.sqrt( dCdT / (1/2*K**2 * varphi))
local_volatility

sqrt(zoo*S_T*(-D + r)*Piecewise((erf(sqrt(2)*(mu_google - tau)/(2*sigma_google))/2 + 1/2, (Abs(arg(sigma_google)) < pi/4) | ((Abs(arg(sigma_google)) <= pi/4) & (Abs(2*arg(mu_google) - 4*arg(sigma_google) + 2*arg((mu_google - tau)/mu_google) + 2*pi) < pi))), (sqrt(2)*Integral(exp(-(_z - mu_google + tau)**2/(2*sigma_google**2)), (_z, 0, oo))/(2*sqrt(pi)*sigma_google), True)))

# Using real values from QuantBook to calculate Local Volatility

In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
import datetime
qb = QuantBook()
option = qb.AddOption("GOOG") 
option.SetFilter(-2, 2, 0, 90)
history = qb.GetOptionHistory(option.Symbol, datetime(2023, 5, 7), datetime(2023, 5, 26))

all_history = history.GetAllData()
expiries = history.GetExpiryDates() 
strikes = history.GetStrikes()

goog = qb.AddEquity("GOOG")
goog_df = qb.History(qb.Securities.Keys, datetime(2023, 5, 7), datetime(2023, 5, 26), Resolution.Daily)
goog_df = goog_df['close'].unstack(level=0)
S_T = goog_df.values[-1]
all_history.head()

In [ ]:
#BSM model of Call option
import math
import sympy.stats

####################### parameters #########################
tau = 3/220 #expiry the 5/19
r = 0.0341 
D = 0.0
K = strikes[-1]
sigma = goog_df.std()
S_T = float(goog_df.values[-1])
mu_goog = goog_df.mean()
#Calculate numerically the local volatility
local_volatility.subs([(S_T,S_T),(K,int(K)),(tau,tau),(D,D),(sigma_google,sigma),(mu_google,mu_google)]).evalf()
